In [ ]:
%load_ext lab_black
%cd ../../..

In [ ]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import DistilBertForSequenceClassification, DistilBertTokenizer

from datasets.dataset_ver8 import DatasetGeneratorVer8
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main_ver8 import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm

In [ ]:
postfix = "distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_3"
outdir = Path("results/distilbert-base-uncased-ver8")
fold = 3

In [ ]:
with open("config/distilbert-base-uncased-ver8.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [ ]:
C

In [ ]:
trainer = MyTrainer(C, fold, outdir / f"{postfix}.pth")

In [ ]:
model = trainer.model
model.eval()
torch.set_grad_enabled(False)

In [ ]:
activation = []


def hook(model, input, output):
    activation.append(output.detach().cpu())

In [ ]:
model.module.pre_classifier.register_forward_hook(hook)

In [ ]:
train_dl = trainer.dsgen.train_only()

# 덱만들기 (Train)

In [ ]:
activation = []
deck = {"feat": [], "otext": [], "tlevel": [], "fclevel": []}
with tqdm(total=len(train_dl.dataset), ncols=100, file=sys.stdout) as t:
    for text, tlevel, otext in train_dl:
        fclevel = model(text.cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["tlevel"].append(tlevel)
        deck["otext"].extend(otext)
        t.update(text.size(0))

In [ ]:
deck["feat"] = torch.cat(activation)
deck["tlevel"] = torch.cat(deck["tlevel"])
deck["fclevel"] = torch.cat(deck["fclevel"])

In [ ]:
deck["tlevel"].shape, deck["fclevel"].shape

In [ ]:
deck["feat"].shape

In [ ]:
torch.save(deck, outdir / f"{postfix}-deck1.pth")

# Valid Lv7에 대한 결과 분석

In [ ]:
deck = torch.load("results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_1-deck1.pth")

In [ ]:
ds_lv7 = trainer.dsgen.valid_lv7().dataset

In [ ]:
text, otext = ds_lv7[0]
activation = []
print(model(text[None].cuda())[0].cpu())

In [ ]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

In [ ]:
text, otext = ds_lv7[1]
activation = []
print(model(text[None].cuda())[0].cpu())

In [ ]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

In [ ]:
text, otext = ds_lv7[2]
activation = []
print(model(text[None].cuda())[0].cpu())

In [ ]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

# 덱만들기 (Submission)

In [ ]:
ds_test = trainer.dsgen.test()

In [ ]:
activation = []
deck = {"feat": [], "otext": [], "fclevel": [], "ids": []}
with tqdm(total=len(ds_test), ncols=100, file=sys.stdout) as t:
    for i in range(len(ds_test)):
        text, otext, ids = ds_test[i]
        fclevel = model(text[None].cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["otext"].append(otext)
        deck["ids"].append(ids)
        t.update()

In [ ]:
deck["feat"] = torch.stack(activation)
deck["fclevel"] = torch.stack(deck["fclevel"])

In [ ]:
deck["feat"] = deck["feat"][:, 0, :]
deck["fclevel"] = deck["fclevel"][:, 0]

In [ ]:
deck["feat"].shape, deck["fclevel"].shape

In [ ]:
torch.save(deck, outdir / f"{postfix}-deck2.pth")